In [1]:
from gpt import gpts

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Writing the prompts for generating facts:
number_of_facts = 30
number_of_questions = 5
prompt = f"""
First, generate {number_of_facts} facts about a character named John and his friends: you can make up the name for his friends on your own. Be sure to be creative with the facts, and generate facts that describe many different aspects of John's life. Number these facts 1 through {number_of_facts} and display them. Surround the facts with a "[START FACTS]" and "[END FACTS]" label.
Then, create {number_of_questions} multiple choice questions revolving around the generated facts that a user who knows John very well should know the answer to, each with 4 choices: one of the choices should be correct, while the other 3 should be incorrect. Surround each question and the choices with a "[START QUESTION]" and "[END QUESTION]" label, and surround the correct answer with a "[START ANSWER]" and "[END ANSWER]" label. 
"""
# response = openai.ChatCompletion.create(
#             model='gpt-4',
#             messages=[{'role': 'user', 'content': prompt}]
#         )
# response_content = response["choices"][0]["message"]["content"]
# response_content

In [23]:
def process_response(response):
    response_content = response
    try:
        # Processing the facts
        response_content = response_content.split("[END FACTS]")
        facts = response_content[0].split("[START FACTS]")[1]
        fact_array = facts.split("\n")
        fact_array = list(filter(None, fact_array))
        for i in range(len(fact_array)):
            fact_array[i] = fact_array[i].split('.')[1][1:]
        
        # Processing the questions
        questions_array = []
        questions = response_content[1].split("[END ANSWER]")
        for question in questions[:-1]:
            question = question.split("[START ANSWER]")
            letter_choice = question[1].strip('\n')
            letter_choice = letter_choice[0]
            question = question[0].split("[START QUESTION]")
            question = question[1].strip('\n')
            question = question.split("[END QUESTION]")[0]
            questions_array.append((question, letter_choice))
        
        return fact_array, questions_array

    except Exception as e:
        print(e)
        print("Error parsing, moving on...") 
        return None, None

# fact_array, questions_array = process_response(response)
# print(fact_array)
# print(questions_array)

In [11]:
# Generate Dataset:
import json
num_points = 100
dataset = []
responses = gpts([prompt]*num_points, model='gpt-4')

100%|██████████| 100/100 [00:44<00:00,  2.27it/s]


In [19]:
# Generating 400 more datapoints:
num_points = 400
responses2 = gpts([prompt]*num_points, model='gpt-4')

100%|██████████| 200/200 [01:41<00:00,  1.97it/s]


In [20]:
responses = responses + responses2

In [26]:
dataset = []
for response in responses: 
    fact_array, questions_array = process_response(response)
    if fact_array and questions_array:
        data = {'facts': fact_array, 'questions': questions_array}
        dataset.append(data)

with open('data/data.json', 'w') as f:
    json.dump(dataset, f)

string index out of range
Error parsing, moving on...
list index out of range
Error parsing, moving on...


In [3]:
from agent import MemoryAgent
from datastore import PineconeDatastore

/Users/benshi/opt/anaconda3/envs/agenv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [10]:
import json
with open('data/data.json', 'r') as f:
    dataset = json.load(f)

dataset_part1 = dataset[:100]
# Evaluate the baseline (with context)
def get_answers(d):
    prompts = []
    answers = []
    for point in d:
        fact_array = point['facts']
        questions_array = point['questions']
        facts_string = '\n'.join(fact_array)
        # Creating the prompts:
        prompt = f"""Here are facts about a person. Given these facts, answer all the given multiple choice question based on the given facts. Respond with just the letter of the correct answer for each question, and nothing else, and separate each answer with a space in between.
        [START FACTS]
        {facts_string}
        [END FACTS]
        [START QUESTION]
        {questions_array[0][0]}
        [END QUESTION]
        [START QUESTION]
        {questions_array[1][0]}
        [END QUESTION]
        [START QUESTION]
        {questions_array[2][0]}
        [END QUESTION]
        [START QUESTION]
        {questions_array[3][0]}
        [END QUESTION]
        [START QUESTION]
        {questions_array[4][0]}
        [END QUESTION]
        """
        prompts.append(prompt)
        answers.append(questions_array[0][1])
        answers.append(questions_array[1][1])
        answers.append(questions_array[2][1])
        answers.append(questions_array[3][1])
        answers.append(questions_array[4][1])

    responses = gpts(prompts, model='gpt-4')
    return responses, answers
responses1 = get_answers(dataset_part1)

100%|██████████| 100/100 [50:49<00:00, 30.50s/it]


In [17]:
%store responses1

Stored 'responses1' (list)


In [34]:
# Grading:
score = 0
bad_count = 0
for r, point in zip(responses1, dataset[:100]):
    model_answers = r.split(' ')
    if len(model_answers) != 5:
        bad_count += 5
        continue
    for i, ans in enumerate(model_answers):
        if '.' in ans:
            model_answers[i] = ans.strip('.')

    questions_array = point['questions']
    for i, question in enumerate(questions_array):
        answer = question[1]
        if not answer or answer == '\n' or answer == ' ':
            bad_count += 1
            continue
        if answer.strip().lower() == model_answers[i].strip().lower():
            score += 1
        # elif len(answer.strip().strip('\n').strip().lower()) != len(model_answers[i].strip().strip('\n').strip().lower()):
        #     print(answer.lower())
        #     print(model_answers[i].lower())
total = 500 - bad_count
percentage_score = score / total * 100
print(percentage_score)

91.11111111111111


In [40]:
for question in dataset[0]['questions']:
    print(question[1])

C
B
B
B
B


In [ ]:
dataset_part2 = dataset[100:200]
responses2 = get_answers(dataset_part2)

In [9]:
print(responses1)

['C B B B B']


In [ ]:
# Evaluate the Agent: 